In [5]:
import nltk
from newspaper import Article
from textblob import TextBlob
nltk.download('punkt')
import pandas as pd
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sunei\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
def print_data(blog_urls): 
    for link1 in blog_urls:

        #keywords
        article = Article(link1)
        article.download()
        article.parse()
        
        # Perform natural language processing on the article
        article.nlp()
        
        # Extract keywords from the article
        keywords = article.keywords
        
        #sentiments
        # Create a TextBlob object from the article text
        text_blob = TextBlob(article.text)
        
        # Get the sentiment polarity of the article
        sentiment_polarity = text_blob.sentiment.polarity
        
        #shares
        
        driver.get(link1)
        try:
            # Wait for the dynamic content to load
            st_label_span = WebDriverWait(driver, 30).until(
                EC.presence_of_element_located((By.CLASS_NAME, "st-label"))
            )
            # Extract the dynamic data
            shares = st_label_span.text.strip()
        except Exception as e:
            shares = 'null'
    
        #headings
        html_text = requests.get(link1).text
        soup = BeautifulSoup(html_text, 'lxml')
        
        if soup.find('h2', class_="heading"):
            head = soup.find('h2', class_="heading").text.strip()
        else:
            head = 'null'
        #dates
        if soup.find('div', class_="post-hour"):
            date = soup.find('div', class_="post-hour").text.strip()
        else:
            date = 'null'
        
        print('head: ',head)
        print('date published: ',date)
        print('shares: ',shares)
        print('keywords:',keywords)
        print('sentiments: ',sentiment_polarity)
        
        # Append the static data along with the dynamic data
        court_office_data.append([head, date, shares,keywords,sentiment_polarity])
        
        print('-----------------------------------------------------------------------------------------------')


In [11]:
def get_all_url(url):
    # Send a GET request to the website
    response = requests.get(url)
    
    # Parse the HTML content of the webpage
    soup = BeautifulSoup(response.content, "html.parser")
    
    # Find all links on the webpage
    links = soup.find_all("a")
    
    # List to store blog post URLs
    blog_urls = []
    
    # Extract URLs of blog posts
    for link in links:
        href = link.get("href")
        if href and "/story/" in href:  # Assuming blog post URLs contain "/story/"
            blog_urls.append(href)

    print_data(blog_urls)


In [12]:
court_office_data = []
driver = webdriver.Chrome()
page_no = 2
url = "https://www.ratopati.com/category/court-office"
while url:
    get_all_url(url)
    url = "https://www.ratopati.com/category/court-office?page={}".format(page_no)
    page_no += 1

# Close the WebDriver
driver.quit()

head:  यसरी समातियो शृङ्खलाबद्ध सहकारी चोरीमा संलग्न ‘फाखेल ग्रुप’
date published:  मङ्गलबार, ११ वैशाख २०८१, १४ : ०४
shares:  
keywords: ['चरम', 'शङखलबदध', 'गरप', 'समतय', 'यसर', 'फखल', 'सहकर', 'सलगन']
sentiments:  0.0
-----------------------------------------------------------------------------------------------
head:  झापामा ब्राउन सुगरसहित एक पक्राउ
date published:  मङ्गलबार, ११ वैशाख २०८१, १३ : २३
shares:  
keywords: ['झपम', 'सगरसहत', 'एक', 'बरउन', 'पकरउ']
sentiments:  0.0
-----------------------------------------------------------------------------------------------
head:  ६ सय ३० विद्यार्थीलाई छात्रवृत्ति
date published:  मङ्गलबार, ११ वैशाख २०८१, ०६ : ४७
shares:  
keywords: ['३०', '६', 'छतरवतत', 'वदयरथलई', 'सय']
sentiments:  0.0
-----------------------------------------------------------------------------------------------
head:  क्यानद्वारा सोलुखुम्बुको महाकुलुङमा प्याराग्लाइडिङको परीक्षणको अनुमति
date published:  सोमबार, १० वैशाख २०८१, २२ : ४४
shares:  20
keywords: ['परकषणक', 'स

ArticleException: Article `download()` failed with HTTPSConnectionPool(host='www.ratopati.com', port=443): Read timed out. (read timeout=7) on URL https://www.ratopati.com/story/389070/government-karmachari

In [14]:
columns = ['headings','date published', 'shares', 'keywords', 'sentiments']
df = pd.DataFrame(court_office_data, columns=columns)
df.to_excel("output.xlsx", index=False)